[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1sd17CQ1Jo5kY_TYQzWdl4Rj0pTQG6oeY?usp=sharing)


# get data

In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv('text2music.csv', index_col='Unnamed: 0')

In [ ]:
data = dataset[dataset['story'] == '']

In [ ]:
texts_base = []
for i in range(data.shape[0]):
  line = ''
  for col in data.columns:
    if col != 'music':
      line += data.iloc[i][col] + ' | '

  texts_base.append(line)

In [ ]:
texts_base[:5]

['Pop | Counting Stars | "Counting Stars" is a song by American pop rock band OneRepublic from their third studio album, Native (2013). The song was written by lead singer Ryan Tedder, and produced by Tedder and Noel Zancanella. It was released as the album\'s second single on June 14, 2013. The song has been one of the band\'s most successful singles, reaching number one in many countries including Canada and the United Kingdom. | Ryan Tedder |  | ',
 'Country | Country Bumpkin | "Country Bumpkin" is a song written by Don Wayne, and recorded by American country music artist Cal Smith. It was released in February 1974 as the first single and title track from the album Country Bumpkins. The single stayed at number one for a single week and spent a total of ten weeks on the country chart. It received Song of the Year Awards from the Academy of Country Music and the Country Music Association. | Don Wayne |  | ',
 'Pop | Cracklin\' Rosie | "Cracklin\' Rosie" is a song written and recorded 

# connect to GPT-3.5

In [ ]:
!pip install openai

/bin/bash: /root/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [ ]:
api_key = ''

In [ ]:
import os
import openai

# Replace "your_api_key_here" with your actual API key or set the environment variable
api_key = os.getenv("OPENAI_API_KEY", api_key)
openai.api_key = api_key

def ask_gpt35(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt }],
        max_tokens = 2000,
        n=5,
        stop=None,
        temperature=0.8,
    )
    message = response.choices[0]['message']['content']
    return message


In [ ]:
from tqdm import tqdm

In [ ]:
prompt = '''Help me to fune-tune model. In list below you can see strings with info about song. Create story about feelings, inspired by each song.
Requirements:
1.Return story for each line in folowing format:

SONG NAME:...
STORY:...

2. You must to divide each song name and story by ###.
3. Avoid mention of sertain song in story, provide only general info in story.

List of songs info:
{lines_list}'''


In [ ]:
answers = []

In [ ]:
len(texts_base)

35

In [ ]:
new_answers = []

In [ ]:
shift = 7
for i in tqdm(range(len(new_answers),len(texts_base) // shift - 1)):
  answer = ask_gpt35(prompt.format(lines_list = texts_base[i*shift:(i+1)*shift]))
  new_answers.append(answer)

100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [04:41<00:00, 70.26s/it]


In [ ]:
len(new_answers)

100

In [ ]:
def get_song_stories(answers):
  song_story = {}
  wrong_answer_list = []
  group_id = 0

  for answer in tqdm(answers):
    answer_list = answer.split('###')
    if len(answer_list) == 1:
      wrong_answer_list.append((group_id, answer))
      group_id += 1
      continue
    for line in answer_list:
      if line == '':
        continue
      story = line.split('STORY: ')[-1].replace('\n','')
      song_name = line.split('STORY:')[0].split('SONG NAME: ')[-1].replace('\n','').lower()
      song_story[song_name] = story
    group_id += 1
  return song_story, wrong_answer_list

In [ ]:
songs_story_dict, wrong_answer_list = get_song_stories(answers)

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 75045.70it/s]


In [ ]:
songs_empty_dict, wrong_answer_list = get_song_stories(new_answers)

100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 34100.03it/s]


In [ ]:
songs_empty_dict.keys()

dict_keys(['counting stars', 'country bumpkin', "cracklin' rosie", 'crazy he calls me', 'crazy little thing called love', 'crazy rhythm', 'crying in the chapel', 'crying my heart out over you', 'crying time', 'cuddle up a little closer, lovey mine', 'for baby (for bobbie)', 'g. i. jive', "i can't give you anything but love", 'no more blues', 'on top of old smoky', 'toot, toot, tootsie', 'total eclipse of the heart', 'try a little tenderness', 'tulips from amsterdam', 'tumbling tumbleweeds', 'turn out the stars', 'turn! turn! turn!', 'turning tables', 'tuxedo junction', 'two sleepy people', "you're so vain", "you're the reason i'm living", "you've changed"])

In [ ]:
for el in wrong_answer_list[1][1].split('\n\n'):
  story = line.split('STORY: ')[-1].replace('\n','')
  song_name = line.split('STORY:')[0].split('SONG NAME: ')[-1].replace('\n','').lower()
  songs_story_dict[song_name] = story

In [ ]:
songs_story_dict["'deed i do"] = songs_story_dict['deed i do']

In [ ]:
def get_story(text):
  text = text.lower()
  if text in songs_story_dict:
    return songs_story_dict[text]
  elif text in songs_empty_dict:
    return songs_empty_dict[text]
  else:
    return ''

In [ ]:
dataset['story'] = dataset['name'].apply(get_story)

In [ ]:
dataset[dataset['story'] == ''].shape

(7, 6)

In [ ]:
dataset.to_csv('text2music.csv')

In [ ]:
dataset

,music,genre,name,text,artist,story
0,X:1\nL:1/8\nQ:1/4=165\nM:4/4\nK:F\nV:1 treble ...,Jazz,'Deed I Do,"""Deed I Do"" is a 1926 jazz standard composed b...",Fred Rose,"As the smooth jazz notes of ""Deed I Do"" filled..."
1,"X:1\nL:1/4\nM:4/4\nK:C\nV:1 treble nm=""Electri...",Country,(Now and Then There's) a Fool Such as I,"""Now and Then There's A Fool Such as I"" is a p...",Bill Trader,"The twang of the country chords in ""Now and Th..."
2,"X:1\nL:1/4\nM:4/4\nK:F\nV:1 treble nm=""Electri...",Folk,(Remember Me) I'm the One Who Loves You,"""(Remember Me) I'm The One Who Loves You"" is ...",Stuart Hamblen,"The gentle strumming of the folk guitar in ""(R..."
3,X:1\nL:1/8\nQ:1/4=216000\nM:4/4\nK:G\nV:1 treb...,R&B,(Sittin' On) the Dock of the Bay,"""The Dock of the Bay"" is a song co-written by ...","Otis Reading, Steve Cropper","The soulful sounds of ""(Sittin' On) the Dock o..."
4,"X:1\nL:1/8\nM:3/4\nK:C\nV:1 treble nm=""Electri...",R&B,(There'll Be) Peace in the Valley,"""There'll Be Peace in the Valley for Me"" is a ...",Thomas A. Dorsey,The gospel-inspired vocals and uplifting lyric...
...,...,...,...,...,...,...
1005,"X:1\nL:1/4\nM:4/4\nK:C\nV:1 treble \nV:1\n z""C...",Jazz,Younger Than Springtime,"""Younger Than Springtime"" is a show tune from ...",Richard Rodgers,
1006,"X:1\nL:1/4\nM:4/4\nK:C\nV:1 treble \nV:1\n z""G...",Country,Your Cheatin' Heart,"""Your Cheatin' Heart"" is a song written and re...",Hank Williams,
1007,X:1\n%%score ( 1 2 )\nL:1/16\nM:4/4\nK:Bb\nV:1...,Rock,Your Song,"""Your Song"" is a song written by Elton John an...",Elton John,
1008,"X:1\nL:1/4\nM:2/2\nK:F\nV:1 treble nm=""Voice"" ...",Pop,Yours Is My Heart Alone,"""Yours Is My Heart Alone"" or ""You Are My Heart...",Franz Lehar,
